# Creating a basic chat experience with kernel arguments

In this example, we show how you can build a simple chat bot by sending and updating arguments with your requests. 

We introduce the Kernel Arguments object which in this demo functions similarly as a key-value store that you can use when running the kernel.  

In this chat scenario, as the user talks back and forth with the bot, the arguments get populated with the history of the conversation. During each new run of the kernel, the arguments will be provided to the AI with content. 

In [ ]:
#r "nuget: Microsoft.SemanticKernel, 1.23.0"
#!import config/Settings.cs

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Kernel = Microsoft.SemanticKernel.Kernel;

var builder = Kernel.CreateBuilder();

// Configure AI service credentials used by the kernel
var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();

if (useAzureOpenAI)
    builder.AddAzureOpenAIChatCompletion(model, azureEndpoint, apiKey);
else
    builder.AddOpenAIChatCompletion(model, apiKey, orgId);

var kernel = builder.Build();

In [1]:
#r "nuget: Microsoft.SemanticKernel, 1.23.0"
#!import config/Settings.cs
#!import config/BuilderForOllama.cs

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Microsoft.SemanticKernel.TemplateEngine;

var kernel = BuilderForOllama.BuildKernel("deepseek-r1");


Installed Packages Microsoft.SemanticKernel, 1.23.0

Using model: deepseek-r1:1.5b, ollama, 


Let's define a prompt outlining a dialogue chat bot.

In [2]:
const string skPrompt = @"
ChatBot can have a conversation with you about any topic.
It can give explicit instructions or say 'I don't know' if it does not have an answer.

{{$history}}
User: {{$userInput}}
ChatBot:";

var executionSettings = new OpenAIPromptExecutionSettings 
{
    MaxTokens = 2000,
    Temperature = 0.7,
    TopP = 0.5
};

Register your semantic function

In [3]:
var chatFunction = kernel.CreateFunctionFromPrompt(skPrompt, executionSettings);

Initialize your arguments

In [4]:
var history = "";
var arguments = new KernelArguments()
{
    ["history"] = history
};

Chat with the Bot

In [5]:
var userInput = "Hi, I'm looking for book suggestions";
arguments["userInput"] = userInput;

var bot_answer = await chatFunction.InvokeAsync(kernel, arguments);

Update the history with the output and set this as the new input value for the next request

In [6]:
history += $"\nUser: {userInput}\nAI: {bot_answer}\n";
arguments["history"] = history;

Console.WriteLine(history);


User: Hi, I'm looking for book suggestions
AI: <think>
Okay, so the user is asking for book suggestions. I need to figure out how to respond appropriately. Since they're just starting out, maybe they don't have much knowledge about books yet. I should keep it simple and friendly.

I remember that sometimes people might not know if a book is good or bad, but in this case, the user didn't specify any preferences or concerns. So, it's probably best to give them a general recommendation without leaning too heavily on genre or author.

I should mention that there are many great books out there and suggest starting with something they can read on their own. Maybe recommend some popular series like "The Great Gatsby" or "To Kill a Mockingbird." That way, the user feels guided to explore different genres.

Also, I don't want to come off as too pushy or dismissive. It's important to encourage them without overstepping into potentially inappropriate territory if they had any concerns. So, just 

Keep Chatting!

In [7]:
Func<string, Task> Chat = async (string input) => {
    // Save new message in the arguments
    arguments["userInput"] = input;

    // Process the user message and get an answer
    var answer = await chatFunction.InvokeAsync(kernel, arguments);

    // Append the new interaction to the chat history
    var result = $"\nUser: {input}\nAI: {answer}\n";
    history += result;

    arguments["history"] = history;
    
    // Show the response
    Console.WriteLine(result);
};

In [8]:
await Chat("I would like a non-fiction book suggestion about Greece history. Please only list one book.");


User: I would like a non-fiction book suggestion about Greece history. Please only list one book.
AI: <think>
Okay, so the user is asking for a non-fiction book about Greece's history. They specifically mentioned "non-fiction," which means it should be written by professionals or experts rather than just someone's opinion.

I need to find a good book that covers the history comprehensively without being too biased. Let me think about some well-known historical books on Greece. 

*The History of Greece* by James R. Brown comes to mind. It's a classic and covers a wide range of topics, including ancient Greek society, culture, and major events like the Peloponnesian War. That sounds perfect because it touches on both ancient and modern aspects.

Another option could be *The Complete Greek Alphabet: A History of the Greek Language* by James R. Brown as well. It provides a detailed look at the language and its development over time, which is another important aspect of history.

I should 

In [9]:
await Chat("that sounds interesting, what are some of the topics I will learn about?");


User: that sounds interesting, what are some of the topics I will learn about?
AI: <think>
Okay, so the user just mentioned "that sounds interesting, what are some of the topics I will learn about?" Hmm, they seem a bit unsure about where to start. They might be new to reading history or literature and want to know which books to read next.

Looking back at our previous conversation, the user asked for non-fiction books on Greece's history. So, we should focus on that category. The AI provided two good recommendations: *The History of Greece* by James R. Brown and *The Complete Greek Alphabet: A History of the Greek Language* by James R. Brown.

I think these are solid choices because they cover both ancient and modern aspects of Greece's history. Maybe the user is looking for a mix of depth and breadth. 

I should also consider if there are other books that might be more specific or accessible. For example, *The Story of Athens* by James R. Brown is another good option since it focus

In [10]:
await Chat("Which topic from the ones you listed do you think most people find interesting?");


User: Which topic from the ones you listed do you think most people find interesting?
AI: <think>
Okay, so I need to figure out which of the topics I mentioned would most likely interest someone looking for book suggestions about Greece's history. The topics include things like ancient civilizations, modern governance, language development, and specific cities like Athens or Rome.

Hmm, let me think about each one. Ancient civilizations are always a big topic, but maybe not everyone has a strong background in history. Modern governance is also important because it shows how the country has changed over time. Language development might be more niche since most people aren't as familiar with Greek. Specific cities like Athens or Rome could be interesting if someone wants to dive deep into one area.

I wonder which ones are more likely to spark curiosity. Maybe ancient civilizations and modern governance? Or perhaps both. I should consider the balance between breadth and depth. If a book

In [11]:
await Chat("could you list some more books I could read about the topic(s) you mentioned?");


User: could you list some more books I could read about the topic(s) you mentioned?
AI: <think>
Okay, so the user just asked for more book suggestions after we discussed ancient civilizations and modern governance. I need to come up with a few good options that cover these topics without being too specific or obscure.

First, I should think about books that are well-regarded in history. Maybe something like *The Story of Ancient Rome* by Michael D. Neubauer would be a good fit since it covers the basics of ancient Rome, which is a significant part of Greece's history.

Next, for modern governance, I remember reading about the Roman Republics and how they influenced later empires. *The Rise and Fall of the Roman Empire: A History* by David W. Macaulay seems like a solid choice because it delves into that period and shows how Rome's ideas carried over to other regions.

For ancient civilizations specifically, I think about *The History of Ancient Egypt* by Richard J. Gibbs is a classic,

After chatting for a while, we have built a growing history, which we are attaching to each prompt and which contains the full conversation. Let's take a look!

In [12]:
Console.WriteLine(history);


User: Hi, I'm looking for book suggestions
AI: <think>
Okay, so the user is asking for book suggestions. I need to figure out how to respond appropriately. Since they're just starting out, maybe they don't have much knowledge about books yet. I should keep it simple and friendly.

I remember that sometimes people might not know if a book is good or bad, but in this case, the user didn't specify any preferences or concerns. So, it's probably best to give them a general recommendation without leaning too heavily on genre or author.

I should mention that there are many great books out there and suggest starting with something they can read on their own. Maybe recommend some popular series like "The Great Gatsby" or "To Kill a Mockingbird." That way, the user feels guided to explore different genres.

Also, I don't want to come off as too pushy or dismissive. It's important to encourage them without overstepping into potentially inappropriate territory if they had any concerns. So, just 